In [20]:
import sqlite3 as sq
import pandas as pd


In [21]:
tv_data = pd.read_csv('../src/america_brazil_2023-08-23_64209.csv')
tv_data

,Ticker,Description,Price,Change %,Change,Technical Rating,Volume,Volume*Price,Market Capitalization,Price to Earnings Ratio (TTM),Basic EPS (TTM),Number of Employees,Sector,Upcoming Earnings Date
0,AALR3,ALLIAR ON NM,10.6900,0.849057,0.0900,Buy,1100.0,1.175900e+04,2.531462e+08,NaN,-0.364263,NaN,Health Services,2023-08-10
1,AALR3L,ALLIAR ON NM,24.5500,NaN,NaN,NaN,36293963.0,8.910168e+08,2.531462e+08,NaN,-0.364263,NaN,Health Services,2023-08-10
2,AALR3F,ALLIAR ON NM,10.8500,-0.549954,-0.0600,Buy,3.0,3.255000e+01,2.531462e+08,NaN,-0.364263,NaN,Health Services,2023-08-10
3,GMGI,"Golden Matrix Group, Inc.",3.2000,-1.234568,-0.0400,Sell,7989.0,2.556480e+04,1.181272e+08,NaN,-0.070200,30.0,Technology Services,2023-08-11
4,CTNM3F,COTEMINAS ON,9.3800,0.000000,0.0000,Sell,1.0,9.380000e+00,3.361928e+07,NaN,NaN,NaN,Consumer Durables,2023-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20177,DMDV,AAM S&P Developed Markets High Dividend Value ETF,21.3725,0.198780,0.0424,Strong Sell,100.0,2.137250e+03,NaN,NaN,NaN,NaN,Miscellaneous,NaN
20178,RVRB,Advisors Series Trust Reverb ETF,22.8900,-1.843911,-0.4300,Sell,101.0,2.311890e+03,NaN,NaN,NaN,NaN,Miscellaneous,NaN
20179,CARD,Bank Of Montreal MAX Auto Industry -3x Inverse...,22.6600,49.669749,7.5200,NaN,100.0,2.266000e+03,NaN,NaN,NaN,NaN,Miscellaneous,NaN
20180,BIB,ProShares Ultra Nasdaq Biotechnology,51.7904,0.058733,0.0304,Buy,300.0,1.553712e+04,NaN,NaN,NaN,NaN,Miscellaneous,NaN


In [22]:
tickers_csv = pd.read_excel('..\src\Copy of NYSE_Stocks.xlsx')
tickers_from_file = tickers_csv['Symbol'].tolist()
ticker_count = len(tickers_from_file)
print(f"Number of unique tickers: {ticker_count}")

Number of unique tickers: 8970


In [23]:
df = pd.DataFrame({'Ticker': tickers_from_file})
df = pd.merge(df, tv_data[['Ticker', 'Market Capitalization', 'Upcoming Earnings Date']], on='Ticker', how='left')
df.rename(columns={'Market Capitalization': 'Mkt Cap', 'Upcoming Earnings Date': 'Next Earnings Date'}, inplace=True)
df


,Ticker,Mkt Cap,Next Earnings Date
0,A,3.474801e+10,2023-11-20
1,AA,5.135780e+09,2023-10-12
2,AAA,NaN,NaN
3,AAAU,NaN,NaN
4,AAC,7.533475e+08,NaN
...,...,...,...
8966,ZVRA,1.730565e+08,2023-11-08
8967,ZWS,4.956683e+09,2023-10-24
8968,ZYME,5.382377e+08,2023-11-07
8969,ZYNE,7.012126e+07,2023-11-13


In [24]:
count_mkt_cap = df['Mkt Cap'].count()
count_earnings_date = df['Next Earnings Date'].count()

total_rows = df.shape[0]

percent_mkt_cap = (count_mkt_cap / total_rows) * 100
percent_earnings_date = (count_earnings_date / total_rows) * 100

print(f"Total de linhas: {total_rows}")
print(f"Valores preenchidos em 'Mkt Cap': {count_mkt_cap} ({percent_mkt_cap:.2f}%)")
print(f"Valores preenchidos em 'Next Earnings Date': {count_earnings_date} ({percent_earnings_date:.2f}%)")

Total de linhas: 8971
Valores preenchidos em 'Mkt Cap': 5605 (62.48%)
Valores preenchidos em 'Next Earnings Date': 4665 (52.00%)


In [26]:
# Connect to SQLite database (will be created if doesn't exist)
conn = sq.connect('../market_data.db')

# Insert data into a table called "mkt_cap_next_earnings"
df.to_sql('mkt_cap_next_earnings', conn, if_exists='replace', index=False)

# Close the database connection
conn.close()